In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
import re
import pickle
import pymysql

#뛰어쓰기 수정
from pykospacing import Spacing
spacing = Spacing()
#맞춤법 수정
from hanspell import spell_checker

import sys
import soynlp
from soynlp.normalizer import *

from krwordrank.word import summarize_with_keywords
from krwordrank.word import KRWordRank

from gensim.models import FastText
FastText_model = FastText.load('Code/FastText_Model')

from konlpy.tag import Mecab
mecab = Mecab(dicpath='C:/mecab/mecab-ko-dic')


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout, Flatten, Activation,BatchNormalization
from tensorflow.keras import Input, Model, layers, optimizers, metrics
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import h5py

import tensorflow as tf

import sys # 시스템
import os  # 시스템


# selenium 크롤링
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time


# 경고 무시
import warnings
warnings.filterwarnings('ignore')
tokenizer = Tokenizer()

In [2]:
food = pd.read_csv('D:/Code/Muchine Learning/TensorFlow/Project/store.csv',encoding='utf-8')
food = food.reset_index(drop=True)
food = food.fillna('')
print(food.isnull().values.any())

False


In [3]:
for i in range(len(food)//10000 + 1):
    if i == (len(food)//10000):
        store_to_csv = food[i * 10000 : len(food)].reset_index(drop=True)
    else:
        store_to_csv = food[i * 10000 : i * 10000 + 10000].reset_index(drop=True)
    print(i * 10000 , i * 10000 + 10000)
    store_name = 'stores_' + str(i) + '.csv'
    store_to_csv.to_csv(store_name, encoding = 'utf-8',index = False)

0 10000
10000 20000
20000 30000
30000 40000
40000 50000
50000 60000
60000 70000
70000 80000
80000 90000
90000 100000
100000 110000
110000 120000
120000 130000
130000 140000
140000 150000


In [2]:
food = pd.read_csv('D:/Code/Muchine Learning/TensorFlow/Project/stores_1.csv',encoding='utf-8')
food = food.reset_index(drop=True)
food = food.fillna('')
print(food.isnull().values.any())

False


In [3]:
food

,id,지번주소,도로명주소,사업장명,좌표정보(X),좌표정보(Y),업태구분명,category_id,table_id
0,10001,서울특별시 송파구 거여동 554-3번지,서울특별시 송파구 오금로 478 (거여동),김밥천국,212493.9123,443538.7679,패스트푸드,etc,food
1,10002,서울특별시 송파구 오금동 73-10번지,서울특별시 송파구 마천로 146 1층 (오금동),족발천국,211913.0508,444680.0797,기타,etc,food
2,10003,서울특별시 송파구 석촌동 177-6번지,서울특별시 송파구 백제고분로39길 16 1층 (석촌동),술있는 식탁 석촌점,209187.0338,444846.4097,기타,etc,food
3,10004,서울특별시 송파구 오금동 11번지,서울특별시 송파구 양재대로72길 17 상가1동 1층 101-1호 (오금동 현대백조아파트),일구칠사'1974',211358.758,445248.7244,기타,etc,food
4,10005,서울특별시 송파구 문정동 644,서울특별시 송파구 법원로11길 11 A동 115호 (문정동 문정현대지식산업센터1-1),앵거스,210398.0,442553.0,기타,etc,food
...,...,...,...,...,...,...,...,...,...
9995,19996,서울특별시 중구 을지로4가 311-1번지,,을지로김밥운동,,,분식,etc,food
9996,19997,서울특별시 중구 신당동 370-51 (지상1층),서울특별시 중구 다산로 155-1 (신당동(지상1층)),망원동 티라미슈,200992.2667,450601.0412,호프/통닭,etc,food
9997,19998,서울특별시 중구 신당동 145-10번지,서울특별시 중구 퇴계로88길 58-4 지하1층 (신당동),볼륨,201675.9513,451261.801,까페,etc,food
9998,19999,서울특별시 중구 신당동 131-5번지 (지상1층),서울특별시 중구 퇴계로84길 17 (신당동(지상1층)),골목길,201628.0898,451407.6249,기타,etc,food


In [4]:
food.isnull().sum()

id             0
지번주소           0
도로명주소          0
사업장명           0
좌표정보(X)        0
좌표정보(Y)        0
업태구분명          0
category_id    0
table_id       0
dtype: int64

In [5]:
tmp = []
str_tmp = ''
cnt = 0
for i, j, k, l in zip(food['사업장명'], food['도로명주소'], food['지번주소'], food['id']):
    tmp_dict = {}
    tmp_dict1 = {}
    if( j != ''):
        # str_tmp = i + ' ' + j
        tmp_dict1['address'] = j
        tmp_dict1['id'] = l
        tmp_dict[i] = tmp_dict1
        tmp.append(tmp_dict)
    elif( k != ''):
        # str_tmp = i + ' ' + k
        tmp_dict1['address'] = k
        tmp_dict1['id'] = l
        tmp_dict[i] = tmp_dict1
        tmp.append(tmp_dict)
    else:
        continue
        

In [6]:

#불용어 정의
stopwords = ['도', '는', '다','것', '그', '들', '수', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임','텐데', '겜', '되', '음', '면']
stop = """
"""

stoplist = re.split(r'[+ + \t + \n]', stop)
# stoplist = re.split(r'[가-힣]', stop)
# stoplist = re.compile('[가-힣]').findall(stop)
for i in range(0, len(stoplist), 3):
    stopwords.append(stoplist[i])
stopwords= [s for s in stopwords if s != '']

# from krwordrank.word import KRWordRank

# min_count = 1   # 단어의 최소 출현 빈도수 (그래프 생성 시)
# max_length = 10 # 단어의 최대 길이

# beta = 0.85   # PageRank의 decaying factor beta
# max_iter = 100


# from krwordrank.word import summarize_with_keywords

# keywords = summarize_with_keywords(texts, min_count=min_count, max_length=max_length,
#     beta=beta, max_iter=max_iter, stopwords=stopwords, verbose=True)

In [7]:
category_words = pd.read_csv('category.csv',encoding='utf-8')
category_words.fillna('', inplace = True)
category_keyword = []


for category in category_words:
    category_tmp = []
    category_tmp.append(category)
    category_keyword.append(category_tmp)

In [8]:
FastText_model.wv.most_similar('매장이 넓어요')

# FastText_model.wv.similarity('신선','청결')

[('왔었어요', 0.7433010935783386),
 ('샀어요', 0.6811431050300598),
 ('였어요', 0.6549246311187744),
 ('했었어요', 0.6416402459144592),
 ('놨어요', 0.6404266953468323),
 ('했어요', 0.6384304165840149),
 ('왔어요', 0.6178849339485168),
 ('어려웠어요', 0.5915448665618896),
 ('거든요', 0.5608712434768677),
 ('겠어요', 0.5559830665588379)]

In [9]:
def sentiment_classify(method_reviews):

    method_texts = method_reviews
    review_tmp = ''
    review_tmp_list = []

    for review in method_texts:
        review_tmp = review_tmp + re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', review)
        if len(review_tmp) > 500:
            review_tmp_list.append(review_tmp)
            review_tmp = ''
    review_tmp_list.append(review_tmp)

    method_texts = []
    for review in review_tmp_list:
        method_texts.append(text_process(review))


    min_count = 1   # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 100 # 단어의 최대 길이

    beta = 0.85  # PageRank의 decaying factor beta
    max_iter = 10

    keywords_tmp = summarize_with_keywords(method_texts, min_count=min_count, max_length=max_length,
        beta=beta, max_iter=max_iter, stopwords=stopwords)
    
    words = sorted(keywords_tmp.items(), key=lambda x:x[1], reverse=True)[:10]
    
    score = {}
    for word in words:
        for category in category_keyword:
            word_cnt = 0
            scores = 0
            score_tmp = []
            for i in category_words[category[0]]:
                if(i != ''):
                    score_tmp.append(FastText_model.wv.similarity(word[0], i))
                    word_cnt += 1
                else:
                    score_tmp.append(0)
            for k in score_tmp:
                scores += k
            scores = scores / word_cnt
            try:
                score[category[0]] += scores
            except:
                score[category[0]] = scores

    for k in score:
        score[k] = float("{:.4f}".format(score[k]))

    score_list = sorted(score.items(), key=lambda x:x[1], reverse=True)[:7]
    
    return score_list

In [10]:
senti_category = {'"혼밥하기 좋아요"': ['가성비', '맛집', '편안한'],
'"매장이 넓어요"': ['술집', '청결', '한적', '편안한', '가족', '편리한'],
'"뷰가 좋아요"' : ['분위기', '청결', '뷰', '기념일', '고급진', '감성적'],
'"단체모임 하기 좋아요"' : ['가성비', '술집', '분위기', '맛집', '시끌벅적', '활동적', '기념일', '고급진', '가족', '모던한'],
'"인테리어가 멋져요"' : ['분위기', '청결', '뷰', '기념일', '고급진', '감성적', '모던한', '이국적', '색다른'],
'"가성비가 좋아요"' : ['가성비', '맛집', '분위기', '편안한'],
'"음식이 맛있어요"' : ['가성비', '맛집', '가족', '고급진', '청결', '색다른', '이국적'],
'"양이 많아요"' : ['가성비', '맛집'],
'"재료가 신선해요"' : ['맛집', '청결', '기념일', '고급진'],
'"특별한 메뉴가 있어요"' : ['분위기', '맛집', '기념일', '고급진', '전통적', '색다른', '이국적'],
'"친절해요"' : ['분위기', '맛집', '기념일', '고급진', '감성적', '편안한'],
'"매장이 청결해요"' : ['청결', '맛집', '한적', '고급진', '모던한'],
'"주차하기 편해요"': ['한적', '가족', '편리한'],
'"특별한 날 가기 좋아요"' : ['분위기', '맛집', '청결', '뷰', '기념일', '고급진', '감성적', '가족', '모던한', '색다른'],
'"화장실이 깨끗해요"' : ['술집', '청결', '고급진', '편리한']
}

In [11]:
def sentiment_classify_keywords(method_dict):
    dict_sentiment_classify_keywords = []
    for senti in method_dict:
        insert_dict_tmp = []
        try:
            method_list_tmp = [senti_category[senti], method_dict[senti] / 500]
            dict_sentiment_classify_keywords.append(method_list_tmp)
        except:
            for method_itr_tmp in sentiment_classify(senti):
                insert_dict_tmp.append(method_itr_tmp[0])
                
            senti_category[senti] = insert_dict_tmp
            method_list_tmp = [senti_category[senti], method_dict[senti] / 500]
            dict_sentiment_classify_keywords.append(method_list_tmp)
    return dict_sentiment_classify_keywords

In [12]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = mecab.morphs(new_sentence) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = 50) # 패딩
    score = float(text_model.predict(pad_new, verbose = 0)) # 예측
    if(score > 0.3):
        return True
    else:
        return False
        
def text_process(text):
    m = re.search('ㅋ+|ㅠ+|ㅜ+', text)
    if m:
        # tmp = spacing(text)
        tmp = spell_checker.check(text)[2]
        tmp = emoticon_normalize(tmp)
        tmp = re.sub('ㅋ+|ㅠ+|ㅜ+', '', tmp)
        tmp = repeat_normalize(tmp, num_repeats=1)
        return tmp

    else :
        tmp = text
    # tmp = spacing(tmp)
    tmp = spell_checker.check(tmp)[2]
    return tmp




In [13]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, values, query): # 단, key와 value는 같음
        # query shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


    
# kargs = {'vocab_size' : vocab_size,
#          'max_len': max_len,
#          'lstm_size' : 64,
#         'dropout_rate' : 0.5,
#         'hidden_dimension' : 250,
#         'output_dimension' : 1}
max_len = 50
vocab_size = 42645
sequence_input = Input(shape=(max_len,))
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len)(sequence_input)

lstm = Bidirectional(LSTM(64, dropout=0.5,  return_sequences = True))(embedded_sequences)

lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)



state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

# hidden = BatchNormalization()(context_vector)
# dense1 = Dense(128, activation="elu")(context_vector)
# dropout_1 = Dropout(0.5)(dense1)
dense2 = Dense(32, activation="elu")(context_vector)
dropout_2 = Dropout(0.5)(dense2)
# dense3 = Dense(32, activation="elu")(dropout_2)
# dropout_3 = Dropout(0.5)(dense3)
dense4 = Dense(16, activation="elu")(dropout_2)
dropout_4 = Dropout(0.5)(dense4)
# dense5 = Dense(8, activation="elu")(dropout_4)
# dropout_5 = Dropout(0.5)(dense5)
output = Dense(1, activation="sigmoid")(dropout_4)
text_model = Model(inputs=sequence_input, outputs=output)

text_model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])


#최적화 과정
modelpath = "./best_model.h5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=2, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

text_model.load_weights('./Model/test_weights.hdf5')

In [14]:
def sql_save(id_check, address, sql_dict_tmp, score, blog_review_cnt):
    date_db = pymysql.connect(host='192.168.0.82', port=3306, user='sky6548', passwd='1234', db='datedb', charset='utf8')
    cursor = date_db.cursor()
    
    # na_sql = """insert into sentiment(`id`) values( (select id from store where `사업장명` = '{0}' and (`도로명주소` = '{1}' or `지번주소` = '{1}')));""".format(id_check, address)
    na_sql = """insert into sentiment(`id`) values( {0} );""".format(id_check)

    cursor.execute(na_sql)

    for sql_dict_tmp_key in sql_dict_tmp:
        sql_dict_tmp_sql = """update sentiment set `{0}` = {1} where `id` = {2};""".format(sql_dict_tmp_key, sql_dict_tmp[sql_dict_tmp_key], id_check)
        cursor.execute(sql_dict_tmp_sql)

    score_sql = """update sentiment set score = {0}, review_cnt = {1} where `id` = {2};""".format(score, blog_review_cnt, id_check)
    cursor.execute(score_sql)
    
    date_db.commit()
    date_db.close()

In [26]:
options = Options()

user_agent= "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument(user_agent)
chrome_path = chromedriver_autoinstaller.install()

driver= webdriver.Chrome(chrome_path, options=options)

In [27]:

# 네이버 지도 검색

for all_cnt in range(0, len(tmp)//10000 + 1):
    # crawling_list = []
    if all_cnt == len(tmp)//10000:
        stores = tmp[all_cnt *10000 : len(tmp)]
    else:
        stores = tmp[all_cnt * 10000 : 10000 + all_cnt * 10000]
        
    crawling_cnt = 0
    
    for store in stores:

        
        store_name_tmp = list(store.keys())[0]
        address = store[store_name_tmp]['address']
        id_check = store[store_name_tmp]['id']
        driver.get("https://map.naver.com/") 
        driver.implicitly_wait(3)
        
        if(crawling_cnt > 500):
            time.sleep(300)
            crawling_cnt=0
        crawling_cnt += 1
        try:
            xpath = "/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input"
            #검색
            search = driver.find_element(By.XPATH,xpath)    
            search.send_keys(list(store.values())[0]['address'])
            search.send_keys(Keys.ENTER)
            driver.implicitly_wait(3)

            try:  
                xpath = "/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-address/div/div[2]/div/div[1]/div[3]/div[2]/div/button"
                driver.find_element(By.XPATH,xpath).click()

                #더보기
                driver.implicitly_wait(3)

                try:
                    #장소 개수
                    xpath = "/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-address/entry-address-detail-place/div[1]/span"
                    cnt = int(driver.find_element(By.XPATH,xpath).text)
                    page = 2
                    store_check = True
                    
                    while(store_check):
                        for i in range(1, 21):
                            #장소 이름
                            xpath = "/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-address/entry-address-detail-place/div[2]/div/div/div[{0}]/div/div[1]".format(i)
                            str_tmp = driver.find_element(By.XPATH,xpath).text

                            store_name = False

                            for store_keys_tmp in list(store.keys())[0].split(' '):
                                if store_keys_tmp in str_tmp:
                                    store_name = True

                            if(store_name):
                                #장소 리뷰 페이지
                                xpath = "/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-address/entry-address-detail-place/div[2]/div/div/div[{0}]".format(i)
                                driver.find_element(By.XPATH,xpath).click()
                                driver.implicitly_wait(3)
    

                                store_num = driver.current_url.split("/place/")[1].split("?c=")[0]


                                review_url = "https://pcmap.place.naver.com/place/{0}/review".format(store_num)
                                driver.get(review_url) 
                                driver.implicitly_wait(3)

                                #키워드 더보기 선택
                                try:
                                    more_keyword = "/html/body/div[3]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/a"
                                    tmp_cnt = 1
                                    while(True):
                                        driver.find_element(By.XPATH,more_keyword).click()
                                        more_keyword = "/html/body/div[3]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/a[{0}]".format(tmp_cnt)
                                        tmp_cnt += 1
                                        driver.implicitly_wait(1)

                                except:
                                    pass
                                #키워드와 키워드 인원     
                                try:
                                    tmp_cnt = 1
                                    keywords = {}
                                    while(True):
                                        xpath = "/html/body/div[3]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/ul/li[{0}]/div[2]/span[1]".format(tmp_cnt)
                                        keyword = driver.find_element(By.XPATH,xpath).text

                                        xpath = "/html/body/div[3]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/ul/li[{0}]/div[2]/span[2]".format(tmp_cnt)
                                        keyword_cnt = driver.find_element(By.XPATH,xpath).text.split("\n")[1]
                                        keywords[keyword] = int(keyword_cnt)
                                        tmp_cnt += 1
                                except:
                                    pass
                                store[list(store.keys())[0]]['keywords'] = keywords


                                # 방문자 리뷰
                                # 리뷰 더보기
                                try:
                                    for i in range(0, 15):
                                        more_review = "/html/body/div[3]/div/div/div/div[7]/div[2]/div[3]/div[2]/a"
                                        driver.find_element(By.XPATH,more_review).click()
                                        driver.implicitly_wait(1)
                                except:
                                    pass


                                try:
                                    reviews = []
                                    review = driver.find_elements(By.CLASS_NAME,'xHaT3')
                                    for text in review:
                                        if len(reviews) > 150:
                                            break
                                        try:
                                            if sentiment_predict(text.text):
                                                reviews.append(text.text)
                                        except:
                                            print('senti error')
                                            pass
                                except:
                                    pass
        
                             # store[list(store.keys())[0]]['reviews'] = reviews
                                keyword_score_list = sentiment_classify_keywords(keywords)
                                review_score_list = sentiment_classify(reviews)
                                sql_dict_tmp = {}

                                for keyword_list_tmp in keyword_score_list:
                                    for keywords_list in keyword_list_tmp[0]:
                                            try:                                    
                                                sql_dict_tmp[keywords_list] = sql_dict_tmp[keywords_list] + keyword_list_tmp[1]
                                            except:
                                                sql_dict_tmp[keywords_list] = keyword_list_tmp[1]
                                                
                                for review_score_tmp in review_score_list:
                                    try:
                                        sql_dict_tmp[review_score_tmp[0]] = sql_dict_tmp[review_score_tmp[0]] + review_score_tmp[1]
                                    except:
                                        sql_dict_tmp[review_score_tmp[0]] = review_score_tmp[1]
                                        
                                for sql_dict_tmp_value in sql_dict_tmp:
                                    sql_dict_tmp[sql_dict_tmp_value] = float("{:.5f}".format(sql_dict_tmp[sql_dict_tmp_value]))
                                    
                                score = -1
                                try:
                                    driver.implicitly_wait(3)
                                    xpath = "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]/em"
                                    score = driver.find_element(By.XPATH,xpath).text
                                    store[list(store.keys())[0]]['score'] = float(score)

                                except:
                                    pass
                                store[list(store.keys())[0]]['score'] = score
                                
                                blog_review_cnt = -1
                                #블로그 리뷰
                                review_url = "https://pcmap.place.naver.com/place/{0}/review/ugc?type=photoView".format(store_num)
                                driver.get(review_url) 
                                driver.implicitly_wait(3)
                                
                                #블로그 리뷰 갯수
                                try:
                                    try:
                                        xpath = "/html/body/div[3]/div/div/div/div[7]/div[2]/div/h2/em"
                                        blog_review_cnt = driver.find_element(By.XPATH,xpath).text
                                    except:
                                        xpath = "/html/body/div[3]/div/div/div/div[6]/div[2]/div/h2/em"
                                        blog_review_cnt = driver.find_element(By.XPATH,xpath).text
                                except:
                                    pass
                                    
                                store[list(store.keys())[0]]['blog_review_cnt'] = blog_review_cnt
                                
                                # print(id_check, address, sql_dict_tmp, score, blog_review_cnt)
                                
                                try:
                                    sql_save(id_check, address, sql_dict_tmp, score, blog_review_cnt)
                                except:
                                    pass
                                
                                #for(1:21) break
                                store_check = false
                                break

                        xpath = "/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-address/entry-address-detail-place/div[3]/pagination-template/div/a[{0}]".format(page)
                        driver.find_element(By.XPATH,xpath).click()
                        page += 1
                        driver.implicitly_wait(3)

                except:
                    pass
            except:
                continue
            # crawling_list.append(store)
        except:
            driver.implicitly_wait(3)
    # str_tmp = 'save_' + str(all_cnt) + '.pkl'
    # f = open(str_tmp, 'wb')
    # pickle.dump(crawling_list, f)
    # f.close()
    
    

KeyboardInterrupt: 

In [ ]:
import pickle
f = open('save.pkl', 'wb')
pickle.dump(tmp[0:10], f)
f.close()

f = open('save.pkl', 'rb')
mydict = pickle.load(f)